In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ozerozdal/Library/Application Support/sagemaker/config.yaml


True

In [2]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = os.environ["BUCKET_NAME"] # get the name of the S3 bucket name from .env file

In [3]:
df = pd.read_csv('data/mobile_price_range_Data.csv')
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [102]:
df.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [4]:
df.shape

(2000, 21)

In [5]:
# 0 (low cost), 1(medium cost), 2 (high cost) and 3 (very high cost)
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [6]:
df.columns.to_list()

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [7]:
# Find the Percentage of Values are missing
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [8]:
features = df.columns.to_list()
label = features.pop(-1)
label

'price_range'

In [9]:
X = df[features]
y = df[label]

In [10]:
X.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [11]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [12]:
X.shape

(2000, 20)

In [13]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [76]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [77]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [78]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
581,1512,1,0.5,0,8,1,18,0.1,88,3,...,1079,1897,3607,12,10,6,1,1,1,3
76,1114,0,2.8,0,4,1,9,0.4,197,3,...,1040,1071,907,10,7,17,1,1,0,0
1916,1176,1,2.1,0,2,1,62,0.5,168,2,...,674,1455,1534,6,2,17,1,0,0,1
1414,1550,1,2.7,0,2,0,32,0.1,126,7,...,1541,1619,1571,12,3,14,0,0,0,2
780,1042,0,2.2,0,15,1,11,0.6,139,5,...,68,1018,2826,18,0,2,1,0,0,2


In [79]:
X_test.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
1860,1646,0,2.5,0,3,1,25,0.6,200,2,5,211,1608,686,8,6,11,1,1,0
353,1182,0,0.5,0,7,1,8,0.5,138,8,16,275,986,2563,19,17,19,1,0,0
1333,1972,0,2.9,0,9,0,14,0.4,196,7,18,293,952,1316,8,1,8,1,1,0
905,989,1,2.0,0,4,0,17,0.2,166,3,19,256,1394,3892,18,7,19,1,1,0
1289,615,1,0.5,1,7,0,58,0.5,130,5,8,1021,1958,1906,14,5,5,1,0,0


In [80]:
y_train

581     3
76      0
1916    1
1414    2
780     2
       ..
1130    3
1294    0
860     2
1459    3
1126    1
Name: price_range, Length: 1700, dtype: int64

In [81]:
y_test

1860    0
353     2
1333    1
905     3
1289    1
       ..
316     1
45      1
1719    0
1831    1
427     3
Name: price_range, Length: 300, dtype: int64

In [82]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [83]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [84]:
trainX.to_csv("outputs/train-V-1.csv", index=False)
testX.to_csv("outputs/test-V-1.csv", index=False)

In [85]:
# send data to S3 bucket
# SageMaker will take training data from S3
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"

train_path = sess.upload_data(
    path='outputs/train-V-1.csv', bucket =bucket, key_prefix=sk_prefix
)

test_path = sess.upload_data(
    path='outputs/test-V-1.csv', bucket =bucket, key_prefix=sk_prefix
)

print(train_path)
print(test_path)

s3://mobprice-aws-sagemaker/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://mobprice-aws-sagemaker/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


In [86]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import joblib
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=42)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, *_ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading Data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = train_df.columns.to_list()
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print("Column order: ")
    print(features)
    print()

    print("Label column is: ", label)
    print()

    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()

    print("Training Random Forest Model......")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [87]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=os.environ["ROLE"],
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 42,
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600
)

In [88]:
# launch training job with asynchronous call
sklearn_estimator.fit({"train": train_path, "test": test_path}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-05-11-17-55-53-016


2024-05-11 17:55:54 Starting - Starting the training job...
2024-05-11 17:56:08 Starting - Preparing the instances for training...
2024-05-11 17:56:37 Downloading - Downloading input data...
2024-05-11 17:57:02 Downloading - Downloading the training image...
2024-05-11 17:57:53 Training - Training image download completed. Training in progress.
2024-05-11 17:57:53 Uploading - Uploading generated training model2024-05-11 17:57:47,989 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-11 17:57:47,992 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-11 17:57:48,036 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-11 17:57:48,184 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-11 17:57:48,196 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-11 17:57:48,207 sagemaker-training-toolkit INFO

In [89]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifacts = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifacts)


2024-05-11 17:58:09 Starting - Preparing the instances for training
2024-05-11 17:58:09 Downloading - Downloading the training image
2024-05-11 17:58:09 Training - Training image download completed. Training in progress.
2024-05-11 17:58:09 Uploading - Uploading generated training model
2024-05-11 17:58:09 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-905418178928/RF-custom-sklearn-2024-05-11-17-55-53-016/output/model.tar.gz


In [90]:
artifacts

's3://sagemaker-us-east-1-905418178928/RF-custom-sklearn-2024-05-11-17-55-53-016/output/model.tar.gz'

In [91]:
from sagemaker.sklearn import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data = artifacts,
    role=os.environ["ROLE"],
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [92]:
model

# Endpoint Deployment

In [93]:
## Endpoint deployment
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_nam=endpoint_name
)

EndpointName=Custom-sklearn-model-2024-05-11-17-59-05


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-05-11-17-58-55
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-05-11-17-58-5-2024-05-11-17-59-07-644
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-05-11-17-58-5-2024-05-11-17-59-07-644


-----!

In [94]:
predictor

In [95]:
endpoint_name

'Custom-sklearn-model-2024-05-11-17-59-05'

In [96]:
# First two rows of X_test and convert them into the list
X_test[features][0:2].values.tolist()

[[1646.0,
  0.0,
  2.5,
  0.0,
  3.0,
  1.0,
  25.0,
  0.6,
  200.0,
  2.0,
  5.0,
  211.0,
  1608.0,
  686.0,
  8.0,
  6.0,
  11.0,
  1.0,
  1.0,
  0.0],
 [1182.0,
  0.0,
  0.5,
  0.0,
  7.0,
  1.0,
  8.0,
  0.5,
  138.0,
  8.0,
  16.0,
  275.0,
  986.0,
  2563.0,
  19.0,
  17.0,
  19.0,
  1.0,
  0.0,
  0.0]]

# Getting predictions for the first 10 rows

In [98]:
# Make predictions using the deployed endpoint
response = predictor.predict(X_test[features][0:10].values.tolist())

# Process the response
print(response)

[0 2 1 3 1 2 2 0 3 1]


# Cleaning - Remove all endpoints

In [100]:
def remove_all_endpoints():
    # Create a SageMaker client
    sagemaker = boto3.client('sagemaker')

    # Get a list of all endpoints
    endpoints_response = sagemaker.list_endpoints()

    # Check if there are any endpoints to delete
    if 'Endpoints' in endpoints_response:
        endpoints = endpoints_response['Endpoints']
        for endpoint in endpoints:
            endpoint_name = endpoint['EndpointName']
            print(f"Deleting endpoint: {endpoint_name}")
            sagemaker.delete_endpoint(EndpointName=endpoint_name)
        print("All endpoints deleted successfully.")
    else:
        print("No endpoints found.")

if __name__ == "__main__":
    remove_all_endpoints()

Deleting endpoint: Custom-sklearn-model-2024-05-11-17-58-5-2024-05-11-17-59-07-644
All endpoints deleted successfully.
